In [78]:
# https://towardsdatascience.com/closures-and-decorators-in-python-2551abbc6eb6

### Scope of variables


Global scope: When a variable is defined outside all functions. A global variable can be accessed by all the functions in the file.

Local scope: When a variable is defined inside a function, it is local to that function. A local variable can be only accessed inside the function in which it was defined.

Nonlocal scope: When a variable is assigned in an enclosing function, it is nonlocal to its nested functions. A nonlocal variable can be accessed by the function in which it was defined and all its nested functions.

In [21]:
x = 1 # x is a global variable  
y = 5 # y is a global variable 
def f():
    # change a global value inside a function, use the "global" keyword.
    global y 
    x = 2   # x is a local variable, globl x is shadowed 
    y += 1  # Reassigning the global variable y
    z = 10   # z is a local variable
    print("Local variable x =", x)
    print("Global variable y =", y)
    print("Local variable z =", z)
f()
print("Global variable  x =", x)
print("Global variable y =", y)

Local variable x = 2
Global variable y = 6
Local variable z = 10
Global variable  x = 1
Global variable y = 6


In [35]:
# In Python, everything is an object, and the variables are references to these objects. 
# When we pass a variable to a function, Python passes a copy of the reference to the object to which the variable refers. 
# It does not send the object or the original reference to the function. So both the original reference and 
# the copied reference that the function receives as its argument are referring to the same object. 
# Now if we pass a global object that is immutable (like an integer or a string), the function cannot modify it using its argument. 
# However, if the object is mutable (like a list), the function can modify it. Here is an example:

a = [1, 2, 3]   # a->[1,2,3]
b = 5           # b->5
def func(x, y): # x->[1,2,3]; y->5
    x.append(4) # x->[1,2,3,4]; a->[1,2,3,4]
    y = y + 1   # y->6; b->5  
    
func(a, b)
print("a=", a)  #  Output is a=[1, 2, 3, 4]
print("b=", b)  #  Output is b=5

a= [1, 2, 3, 4]
b= 5


In [38]:
# nested function. The local variables of the outer function are nonlocal to its inner function.
# The inner function can access the nonlocal variables but cannot change them. 
# Reassigning them simply creates a new local variable with the same name in the inner function "z", 
# and does not affect the nonlocal variable.
# if we want to make a change to a nonlocal variable in a nested function, we must use the nonlocal keyword "y".
# he outer function parameters behave like nonlocal variables and can be also accessed (or read) by the inner function.
def f(x): # outer function 
    y = 5
    z = 10
    t = 10
    def g(): # inner function
        nonlocal y #
        y += 1
        z = 20
        print("Nonlocal variable x =", x)
        print("Local variable z =", z) 
    print("Local variable t =", t)    
    g()
    print("Nonlocal variable x =", x)
    print("Nonlocal variable y =", y)
    print("Local variable z =", z)
f(5)
# This does not work:
# g()

Local variable t = 10
Nonlocal variable x = 5
Local variable z = 20
Nonlocal variable x = 5
Nonlocal variable y = 6
Local variable z = 10


### Closure
Closures make it possible to call an inner function outside the outer function and still access its nonlocal variables.
The name “closure” comes from the fact that it captures the bindings of its free (nonlocal) variables and is the result of closing an open term.

So a closure is an open term which is closed by capturing the bindings of its free (nonlocal) variables. 

1 - It should be returned by the outer function.

2 - it should capture some of the nonlocal variables of the outer function. This can be done by accessing those variables, or defining them as a nonlocal variable or having a nested closure that needs to capture them.

In [40]:
# after running h=f(), all the local variables of f() are gone, and we cannot access x and y anymore. 
# But we still have the value of x that it is returned and stored in h. 
def f():
    x = 5
    y = 10
    return x
h=f()
print(h)

5


In [41]:
# we can make the outer function to return the inner function. 
# This is possible in Python since Python functions are first class. 
# It means that Python treats functions as values, so we can assign a function to a variable, 
# pass it as a function argument or return it by another function. 
# here the outer function f(x) return its inner function g.
def f(x):
    def g(y):
        return y
    return g # not "return g(y)"
a = 5
b = 1
h=f(a) # h->g
h(b)  # Output is 1

1

In [42]:
h.__name__   

'g'

In [43]:
# A function that only takes one argument is called a unary function.
def f(x):
    def g(y):
        return y
    return g
a = 5
b = 1
f(a)(b)  # Output is 1

1

In [44]:
# three nested unary functions. The first function f(x) has an inner function g(y) and g(y) has an inner function h(z). 
# Each outer function returns its inner function.
def f(x):
    def g(y):
        def h(z):
            return z
        return h
    return g
a = 5
b = 2
c = 1
f(a)(b)(c)  # Output is 1

1

In [45]:
# A closure is an inner function with an extended scope that encompasses nonlocal variables of the outer function. 
def f(x):
    z = 2
    # A variable like y which is in the local scope of the inner function g(y) is called a "bound variable"
    # A function that only has bound variables is called a "close term".
    # a nonlocal variable like z is called a "free variable" since it is free to be defined outside g(y), 
    # and a function that contains free variables is called an "open term."
    def g(y): # closure
        return z*x + y
    return g
a = 5
b = 1
h = f(a)
h(b)  # Output is 11

# the inner function g(y) is not a closure as long as it has a free variable which is not bound yet (x and z). 
# Once we evaluate h=f(a), the enclosing function f(x) is evaluated, and the free variables x and z become bound to 5 and 2 respectively. 
# So g(y) returned by f(a) becomes a closure, and h is now referring to a closure

11

In [46]:
# free variables are captured by the inner function. 
h.__code__.co_freevars

('x', 'z')

In [47]:
# also get the value of these free variables using the closure attribute:

print(h.__code__.co_freevars[0], "=", h.__closure__[0].cell_contents) 
print(h.__code__.co_freevars[1], "=", h.__closure__[1].cell_contents)

x = 5
z = 2


In [48]:
# It is important to note that to have a closure, the inner function should access the nonlocal variables of the outer function. 
# When no free variable is accessed inside the inner function, 
# it does not capture them since it is already a closed term and does not need to be closed.

def f(x):
    z = 2
    def g(y):
        return y
    return g
a = 5
b = 1
h = f(a) # not a closure
h(b)  # Output is 1
# the nonlocal variables x and z are not accessed inside g(y) and there is no need for g(y) to capture them

1

In [49]:
h.__code__.co_freevars

()

In [52]:
print(h.__closure__)

None


In [54]:
# If we do not access a nonlocal variable but define it as nonlocal inside the inner function, 
# it is still captured by the closure. 
def f(x):
    z = 2
    t = 3
    def g(y): # closure
        nonlocal t
        return y
    return g
a = 5
b = 1
h = f(a)
h(b)  


1

In [55]:
h.__code__.co_freevars  # Output is ('t',)

('t',)

In [56]:
# here g(y) is not a closure here since the value of x is just used to initialize y and g does not need to capture x.
def f(x):
    def g(y = x):
        return y
    return g
a = 5
b = 1
h = f(a)
h()  # Output is 5

5

In [57]:
h.__code__.co_freevars 

()

In [59]:
# multiple nested functions, each closure is able to capture all the nonlocal variables which are at higher levels
def f(x):
    def g(y):
        def h(z):
            return x * y * z
        return h
    return g
a = 5
b = 2
c = 1
f(a)(b)(c)  # Output is 10

10

In [60]:
# f(a)(b) refers to h
f(a)(b).__code__.co_freevars

('x', 'y')

In [61]:
# g(y) is also a closure and it captures x as a nonlocal variable. 
# We can easily check it (remember that f(a) refers to g(y)):
f(a).__code__.co_freevars  # Output is ('x',)

('x',)

In [ ]:
# g(y) is not a closure, because h(z) does not need to capture x. 
# As a result, g(y) doesn't capture it either, and doesn't become a closure. 
def f(x):
    def g(y): 
        def h(z):
            return y * z
        return h
    return g
a = 5
b = 2
c = 1
f(a).__code__.co_freevars  # Output is ()

In [63]:
# Listing 15
def f(x):
    z = 2
    return lambda y: z*x+y
a = 5
b = 1
f(a)(b)  # Output is 11

11

In [66]:
# class vs function:
class NthRoot:
    def __init__(self, n=2):
        self.n = n
    def set_root(n):
        self.n = n
    def calc(self, x):
        return x ** (1/self.n)
    
thirdRoot = NthRoot(3)
print(thirdRoot.calc(27))  # Output is 3

# functional programming, closures make it possible to bind data to a function without actually passing them as parameters.
def nth_root(n=2):
    def calc(x):
        return x ** (1/n)
    return calc

third_root = nth_root(3)
print(third_root(27))  # Output is 3

3.0
3.0


#### Composition
If we have two functions f and g, we can combine them in such a way so that the output of f becomes the input of g. In mathematics, this operation is called a composition. 

In [67]:
def compose(g, f):
    def h(*args, **kwargs):
        return g(f(*args, **kwargs))
    return h


In [70]:
inch_to_foot= lambda x: x/12
foot_meter= lambda x: x * 0.3048

inch_to_meter = compose(foot_meter, inch_to_foot)
inch_to_meter(12)   # Output 0.3048

0.3048

#### Partial application
In mathematics, the number of arguments that a function takes is called the arity of that function. Partial application is an operation that reduces the arity of a function. It means it allows you to fix the values of some of the arguments and freeze them to get a function with fewer parameters. So it somehow simplifies the function.

For example, the arity of f(x, y, z) is 3. We can fix the value of argument x at a to have f(x=a, y ,z) = g(y,z). Now the arity of g(y,z)is 2, and it is the result of partial application of f(x, y, z). So partial(f) => g. We can implement the partial application using closures:

In [72]:
def partial(f, *f_args, **f_keywords):
    def g(*args, **keywords):
        new_keywords = f_keywords.copy()
        new_keywords.update(keywords)
        return f(*(f_args + args), **new_keywords)
    return g

In [74]:
func = lambda x,y,z: x**2 + 2*y + z

pfunc = partial(func, 1)
pfunc(2, 3)  # Output is 8

8

#### Currying
In mathematics, currying means transforming a function with multiple parameters into a sequence of nested unary functions. As mentioned before a unary function is a function that only takes one argument. So for example, if we have a function f(x, y, z). Currying transforms it to g(x)(y)(z) = ((g(x))(y))(z). 

In [75]:
def curry(f):
    argc = f.__code__.co_argcount
    f_args = []
    f_kwargs = {}
    def g(*args, **kwargs):
        nonlocal f_args, f_kwargs
        f_args += args
        f_kwargs.update(kwargs)
        if len(f_args)+len(f_kwargs) == argc:
            return f(*f_args, **f_kwargs)
        else:
            return g          
    return g

In [76]:
cfunc = curry(func)
cfunc(1)(2)# Output:
# <function __main__.curry.<locals>.g(*args, **kwargs)>

<function __main__.curry.<locals>.g(*args, **kwargs)>

In [77]:
cfunc(3)  # Output is 8

8

### Decoration and decorators

In [18]:
# When we define a function in Python, the name of that function is simply 
# a reference to the body of the function (function definition). 
def f():
    return("f definition")
def g():
    return("g definition")

print("f is referring to ", f())
print("g is referring to ", g())
print(id(f),id(g))

f is referring to  f definition
g is referring to  g definition
2361010188144 2361010187712


In [19]:
print("Swapping f and g")
temp = f
f = g
g = temp

print("f is referring to ", f())
print("g is referring to ", g())
print(id(f),id(g))

Swapping f and g
f is referring to  g definition
g is referring to  f definition
2361010187712 2361010188144
